In [1]:
import os
import user_interaction
import normalize
import pandas as pd

In [ ]:
crime_df = pd.read_csv(os.path.join(os.getcwd(), 'crime.csv'))
crime_df = crime_df.drop(columns=crime_df.columns[10:])
crime_df = crime_df.dropna()
crime_df = crime_df.drop_duplicates()

In [15]:
pollution_df = pd.read_csv(os.path.join(os.getcwd(), 'pollution.csv'))
pollution_df = pollution_df.drop(columns=pollution_df.columns[9:])
pollution_df_small = pollution_df.drop(range(1000000, 1746661))
pollution_df_small = pollution_df_small.drop(columns=['Date Local'])
pollution_df_small = pollution_df_small.drop_duplicates()

In [ ]:
food_df = pd.read_csv(os.path.join(os.getcwd(), 'food.csv'), encoding='latin1')
food_df = food_df.drop(columns=food_df.columns[10:])

In [ ]:
liquor_df = pd.read_csv(os.path.join(os.getcwd(), 'liquor.csv'))
liquor_df = liquor_df.drop(columns=liquor_df.columns[12:])
liquor_df = liquor_df.drop(range(3000000, 12591077))
liquor_df = liquor_df.dropna()
liquor_df = liquor_df.drop_duplicates()

In [16]:
import time
start = time.time()
deps = user_interaction.find_dependencies(pollution_df_small, 0.95, 0.85)
time.time() - start


100%|██████████| 8/8 [04:02<00:00, 33.50s/it]

244.6549677848816

In [17]:
print(deps)

 --> Unnamed: 0
 {State}  {Site Num,County Code}  {Site Num,City}  {Unnamed: 0,County Code}  {County Code,City}  {County}  {Address}  --> State Code
 {State Code,City}  {State,City}  {Site Num,City}  {County}  {Address}  --> County Code
 {Address}  --> Site Num
 {Site Num,City}  {County,Site Num}  {Site Num,County Code}  --> Address
 {State Code}  {Site Num,County Code}  {Site Num,City}  {Unnamed: 0,County Code}  {County Code,City}  {County}  {Address}  --> State
 {State Code,City}  {State,City}  {Site Num,County Code}  {State Code,County Code}  {Site Num,City}  {State,County Code}  {Unnamed: 0,County Code}  {County Code,City}  {Address}  --> County
 {County,Site Num}  {Site Num,County Code}  {Address}  --> City


In [18]:
deps_dic = deps.serialize()

In [22]:
print(deps_dic)

{'Unnamed: 0': [], 'State Code': [['State']], 'County Code': [['County']], 'Site Num': [['Address']], 'Address': [['Site Num', 'City'], ['County', 'Site Num']], 'State': [['County']], 'County': [['Site Num', 'City'], ['Address']], 'City': [['County', 'Site Num'], ['Address']]}


In [21]:
print(deps)
deps.remove_implied_extroneous()
print("\n\n")
print(deps)

 --> Unnamed: 0
 {State}  --> State Code
 {County}  --> County Code
 {Address}  --> Site Num
 {Site Num,City}  {County,Site Num}  --> Address
 {County}  --> State
 {Site Num,City}  {Address}  --> County
 {County,Site Num}  {Address}  --> City



 --> Unnamed: 0
 {State}  --> State Code
 {County}  --> County Code
 {Address}  --> Site Num
 {Site Num,City}  {County,Site Num}  --> Address
 {County}  --> State
 {Site Num,City}  {Address}  --> County
 {County,Site Num}  {Address}  --> City


In [19]:
new_grps = normalize.normalize(deps)

In [20]:
for grp in new_grps:
    print(grp)
    print("\n\n")

 --> Unnamed: 0
 {Address}  --> Site Num
 {Site Num,City}  {County,Site Num}  --> Address
 {Site Num,City}  {Address}  --> County
 {County,Site Num}  {Address}  --> City



 --> County
 {County}  --> County Code
 {County}  --> State



 --> State
 {State}  --> State Code





In [10]:
print(new_grps[0].find_trans_deps())
print(new_grps[0].find_partial_deps())
print(new_grps[0].find_candidate_keys())

[]
[]
[{'County', 'Unnamed: 0', 'Site Num', 'Date Local'}, {'Unnamed: 0', 'City', 'Date Local'}, {'Unnamed: 0', 'Address', 'Date Local'}, {'Unnamed: 0', 'Site Num', 'County Code', 'Date Local'}]


In [16]:
for grp in new_grps:
    print(grp)
    print("\n\n")

 --> Unnamed: 0
 --> City
 --> Date Local



 --> City
 {City}  --> Address
 {City}  --> State



 --> State
 {State}  --> State Code



 --> Address
 {Address}  --> Site Num
 {Address}  --> County



 --> County
 {County}  --> County Code





### Food Example Compacted

In [ ]:
deps = user_interaction.find_dependencies(food_df, 1.00)
print(deps)
deps = user_interaction.find_dependencies(food_df, .95)
print(deps)

In [ ]:
auto_norm_dfs = user_interaction.auto_normalize(food_df)
for df in auto_norm_dfs:
    print(df)

deps = user_interaction.find_dependencies(food_df)
print(deps)
new_grps = normalize.normalize(deps)
for x in new_grps:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ \n')
    print(x)

# Crime Example 

In [ ]:
auto_crime = user_interaction.auto_normalize(crime_df)

In [ ]:
crime_deps = user_interaction.find_dependencies(crime_df)
print(crime_deps)

In [ ]:
new_grps_crime = normalize.normalize(crime_deps)
for x in new_grps_crime:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ \n')
    print(x)

# Liquor Example

In [ ]:
liquor_deps = user_interaction.find_dependencies(liquor_df)
print(liquor_deps)

In [ ]:
new_grps_liquor = normalize.normalize(liquor_deps)
for x in new_grps_liquor:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~  \n')
    print(x)